# Preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv("c:/users/fre_f/ARVOproject/BLclinicalremoved.csv", index_col = "ID", encoding="ansi")
#df1.head()

In [3]:
df1.dtypes

MeiboODinf-0       float64
MeiboOSinf-0       float64
Kjønn M=0          float64
Alder              float64
Synskorr.kir       float64
                    ...   
Skjermbruk-0       float64
Kontaktlinser-0    float64
Mascara-0          float64
Eyeliner-0         float64
AC-0               float64
Length: 92, dtype: object

In [4]:
df1["Vinter"].unique()
df1["RensVarme"].unique()

array(['0', '2', '7', '4', '5', '6', '3', '1', '8', '9', '10', ' ', nan],
      dtype=object)

In [5]:
#Remove empty strings
df1['Vinter'] = df1['Vinter'].replace(" ", np.nan)
df1['RensVarme'] = df1['RensVarme'].replace(" ", np.nan)

In [6]:
df1["Vinter"].unique()
df1["RensVarme"].unique()

array(['0', '2', '7', '4', '5', '6', '3', '1', '8', '9', '10', nan],
      dtype=object)

In [7]:
#Convert dtype
df1['RensVarme']=df1['RensVarme'].astype('float')
df1['Vinter']=df1['Vinter'].astype('float')

In [8]:
df1['MeiboODinf-0'].unique()


array([ 2.,  4.,  3.,  1., nan,  0.])

In [9]:
df1["MeiboODinf-0"].value_counts()

2.0    215
3.0    183
4.0    129
1.0     44
0.0      2
Name: MeiboODinf-0, dtype: int64

In [10]:
#Implement class 0 in class 1
df1['MeiboODinf-0'] = df1['MeiboODinf-0'].replace(0, 1)
df1['MeiboOSinf-0'] = df1['MeiboOSinf-0'].replace(0, 1)

In [11]:
df1.shape

(594, 92)

In [12]:
df1 = df1.dropna(axis=0, subset=["MeiboODinf-0"])
df1 = df1.dropna(axis=0, subset=["MeiboOSinf-0"])

In [13]:
df1.shape

(573, 92)

In [14]:
df1['MeiboODinf-0'].unique()
df1['MeiboOSinf-0'].unique()

array([2., 4., 3., 1.])

In [15]:
#df1.iloc[:,2:92]

In [16]:
#imp = SimpleImputer(strategy='mean')
#imp = IterativeImputer(random_state=42, max_iter=100)
imp = KNNImputer(n_neighbors=3, weights="uniform")
df_imputed = imp.fit_transform(df1)
df2 = pd.DataFrame(df_imputed, columns = df1.columns.tolist(), index = df1.index)
#df2

In [17]:
type(df2)

pandas.core.frame.DataFrame

In [18]:
#df2.iloc[:,0:47]

In [19]:
#df2.iloc[:,0]

In [20]:
#df2.iloc[:,2:] 

In [21]:
X = df2.iloc[:,2:]


In [22]:
y = df2.iloc[:,0]

In [23]:
#y_train for class 1
y_Class1OD = y.replace(2, 0)
y_Class1OD = y_Class1OD.replace(3, 0)
y_Class1OD = y_Class1OD.replace(4, 0)

In [24]:
y_Class1OD.value_counts()

0.0    527
1.0     46
Name: MeiboODinf-0, dtype: int64

In [25]:
y.value_counts()

2.0    215
3.0    183
4.0    129
1.0     46
Name: MeiboODinf-0, dtype: int64

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y_Class1OD, test_size = 0.2, random_state=42)


In [27]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(458, 90)
(115, 90)
(458,)
(115,)


In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trainScaled = scaler.fit_transform(X_train)
X_testScaled = scaler.transform(X_test)
X_trainScaled = pd.DataFrame(X_trainScaled, columns = X_train.columns.tolist(), index=X_train.index)
X_testScaled = pd.DataFrame(X_testScaled, columns = X_test.columns.tolist(), index=X_test.index)

In [29]:
print(X_trainScaled.shape)
print(X_testScaled.shape)
print(y_train.shape)
print(y_test.shape)

(458, 90)
(115, 90)
(458,)
(115,)


In [30]:
def metrics():
    y_pred = clf.predict(X_testScaled)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    cf = confusion_matrix(y_test, y_pred)
    target_names = ['class 0', 'class 1']
    print(f"Balanced accuracy: {bal_acc}")
    print(f"MCC: {mcc}")
    print(f"F1: {f1}")
    print(f"Confusion matrix: \n{cf}")
    print(f"Classification report: \n{classification_report(y_test, y_pred, target_names=target_names)}")

# Training and predicting on the right eye

In [31]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy="most_frequent")
clf.fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()


Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48660714285714285
Confusion matrix: 
[[109   0]
 [  6   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      1.00      0.97       109
     class 1       0.00      0.00      0.00         6

    accuracy                           0.95       115
   macro avg       0.47      0.50      0.49       115
weighted avg       0.90      0.95      0.92       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
from sklearn.svm import LinearSVC
clf = LinearSVC(max_iter=5000).fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.6253822629969419
MCC: 0.1896015841690225
F1: 0.5871306268986468
Confusion matrix: 
[[100   9]
 [  4   2]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.96      0.92      0.94       109
     class 1       0.18      0.33      0.24         6

    accuracy                           0.89       115
   macro avg       0.57      0.63      0.59       115
weighted avg       0.92      0.89      0.90       115



In [33]:
clf = RandomForestClassifier().fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48660714285714285
Confusion matrix: 
[[109   0]
 [  6   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      1.00      0.97       109
     class 1       0.00      0.00      0.00         6

    accuracy                           0.95       115
   macro avg       0.47      0.50      0.49       115
weighted avg       0.90      0.95      0.92       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
clf = RandomForestClassifier(n_estimators=5000, criterion="gini", bootstrap=True, oob_score=False, warm_start=False, random_state=42).fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48660714285714285
Confusion matrix: 
[[109   0]
 [  6   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      1.00      0.97       109
     class 1       0.00      0.00      0.00         6

    accuracy                           0.95       115
   macro avg       0.47      0.50      0.49       115
weighted avg       0.90      0.95      0.92       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=1000, bootstrap=True, oob_score=True, random_state=42).fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48660714285714285
Confusion matrix: 
[[109   0]
 [  6   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      1.00      0.97       109
     class 1       0.00      0.00      0.00         6

    accuracy                           0.95       115
   macro avg       0.47      0.50      0.49       115
weighted avg       0.90      0.95      0.92       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
neigh = KNeighborsClassifier(n_neighbors=3)
clf = neigh.fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.6574923547400612
MCC: 0.38228465101078163
F1: 0.6863636363636364
Confusion matrix: 
[[107   2]
 [  4   2]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.96      0.98      0.97       109
     class 1       0.50      0.33      0.40         6

    accuracy                           0.95       115
   macro avg       0.73      0.66      0.69       115
weighted avg       0.94      0.95      0.94       115



In [37]:
# lbfgs, sgd, adam
#logistic, tanh, relu
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver="lbfgs", activation="relu", learning_rate="constant", random_state=42, max_iter=3000, verbose=False).fit(X_trainScaled, y_train)
#clf.out_activation_="softmax"
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.7782874617737003
MCC: 0.3576308054927703
F1: 0.6481643356643356
Confusion matrix: 
[[97 12]
 [ 2  4]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.98      0.89      0.93       109
     class 1       0.25      0.67      0.36         6

    accuracy                           0.88       115
   macro avg       0.61      0.78      0.65       115
weighted avg       0.94      0.88      0.90       115



In [38]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=50, random_state=42).fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()
#complete_output()

Balanced accuracy: 0.731651376146789
MCC: 0.43091623491635767
F1: 0.7146401985111663
Confusion matrix: 
[[105   4]
 [  3   3]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.97      0.96      0.97       109
     class 1       0.43      0.50      0.46         6

    accuracy                           0.94       115
   macro avg       0.70      0.73      0.71       115
weighted avg       0.94      0.94      0.94       115



In [39]:
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier().fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()
#complete_output()

Balanced accuracy: 0.75
MCC: 0.6975723208794503
F1: 0.8265460030165912
Confusion matrix: 
[[109   0]
 [  3   3]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.97      1.00      0.99       109
     class 1       1.00      0.50      0.67         6

    accuracy                           0.97       115
   macro avg       0.99      0.75      0.83       115
weighted avg       0.97      0.97      0.97       115



In [40]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0, max_depth=None, random_state=42).fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.6162079510703364
MCC: 0.1632235998055146
F1: 0.5697181341980544
Confusion matrix: 
[[98 11]
 [ 4  2]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.96      0.90      0.93       109
     class 1       0.15      0.33      0.21         6

    accuracy                           0.87       115
   macro avg       0.56      0.62      0.57       115
weighted avg       0.92      0.87      0.89       115



In [41]:
from xgboost import XGBClassifier
clf = XGBClassifier(max_depth=10, eta=0.3, reg_lambda=50, alpha=0).fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48660714285714285
Confusion matrix: 
[[109   0]
 [  6   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      1.00      0.97       109
     class 1       0.00      0.00      0.00         6

    accuracy                           0.95       115
   macro avg       0.47      0.50      0.49       115
weighted avg       0.90      0.95      0.92       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
clf = LGBMClassifier().fit(X_trainScaled, y_train)
clf.predict(X_testScaled)

metrics()

Balanced accuracy: 0.7454128440366973
MCC: 0.595695985312917
F1: 0.7909090909090909
Confusion matrix: 
[[108   1]
 [  3   3]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.97      0.99      0.98       109
     class 1       0.75      0.50      0.60         6

    accuracy                           0.97       115
   macro avg       0.86      0.75      0.79       115
weighted avg       0.96      0.97      0.96       115



# Training and predicting on the left eye

In [43]:
#df2.iloc[: ,1]

In [44]:
X_OS = df2.iloc[:,2:]


In [45]:
#df2.iloc[:,1]

In [46]:
y_OS = df2.iloc[:,1]

In [47]:
#y_train for class 1
y_Class1OS = y_OS.replace(2, 0)
y_Class1OS = y_Class1OS.replace(3, 0)
y_Class1OS = y_Class1OS.replace(4, 0)

In [48]:
y_Class1OS.value_counts()

0.0    538
1.0     35
Name: MeiboOSinf-0, dtype: int64

In [49]:
X_trainOS, X_testOS, y_trainOS, y_testOS = train_test_split(X_OS, y_Class1OS, test_size = 0.2, random_state=42)


In [50]:
scaler = StandardScaler()
X_trainScaledOS = scaler.fit_transform(X_trainOS)
X_testScaledOS = scaler.transform(X_testOS)
X_trainScaledOS = pd.DataFrame(X_trainScaledOS, columns = X_trainOS.columns.tolist(), index=X_trainOS.index)
X_testScaledOS = pd.DataFrame(X_testScaledOS, columns = X_testOS.columns.tolist(), index=X_testOS.index)

In [51]:
type(X_trainScaledOS)
type(X_testScaledOS)

pandas.core.frame.DataFrame

In [52]:
print(X_trainScaledOS.shape)
print(X_testScaledOS.shape)
print(y_trainOS.shape)
print(y_testOS.shape)

(458, 90)
(115, 90)
(458,)
(115,)


In [53]:
#X_trainScaledOS

In [54]:
def metricsOS():
    y_predOS = clf.predict(X_testScaledOS)
    bal_acc = balanced_accuracy_score(y_testOS, y_predOS)
    mcc = matthews_corrcoef(y_testOS, y_predOS)
    f1 = f1_score(y_testOS, y_predOS, average="macro")
    cf = confusion_matrix(y_testOS, y_predOS)
    target_names = ['class 0', 'class 1']
    print(f"Balanced accuracy: {bal_acc}")
    print(f"MCC: {mcc}")
    print(f"F1: {f1}")
    print(f"Confusion matrix: \n{cf}")
    print(f"Classification report: \n{classification_report(y_testOS, y_predOS, target_names=target_names)}")

In [55]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy="most_frequent")
clf.fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48198198198198194
Confusion matrix: 
[[107   0]
 [  8   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.93      1.00      0.96       107
     class 1       0.00      0.00      0.00         8

    accuracy                           0.93       115
   macro avg       0.47      0.50      0.48       115
weighted avg       0.87      0.93      0.90       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
from sklearn.svm import LinearSVC
clf = LinearSVC(max_iter=5000).fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.6314252336448598
MCC: 0.19856376833012096
F1: 0.5893719806763285
Confusion matrix: 
[[95 12]
 [ 5  3]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      0.89      0.92       107
     class 1       0.20      0.38      0.26         8

    accuracy                           0.85       115
   macro avg       0.57      0.63      0.59       115
weighted avg       0.90      0.85      0.87       115



In [57]:
clf = RandomForestClassifier().fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48198198198198194
Confusion matrix: 
[[107   0]
 [  8   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.93      1.00      0.96       107
     class 1       0.00      0.00      0.00         8

    accuracy                           0.93       115
   macro avg       0.47      0.50      0.48       115
weighted avg       0.87      0.93      0.90       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
clf = RandomForestClassifier(n_estimators=5000, criterion="gini", bootstrap=True, oob_score=False, warm_start=True, random_state=42).fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48198198198198194
Confusion matrix: 
[[107   0]
 [  8   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.93      1.00      0.96       107
     class 1       0.00      0.00      0.00         8

    accuracy                           0.93       115
   macro avg       0.47      0.50      0.48       115
weighted avg       0.87      0.93      0.90       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
neigh = KNeighborsClassifier(n_neighbors=3)
clf = neigh.fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.49065420560747663
MCC: -0.03637716757892587
F1: 0.47727272727272724
Confusion matrix: 
[[105   2]
 [  8   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.93      0.98      0.95       107
     class 1       0.00      0.00      0.00         8

    accuracy                           0.91       115
   macro avg       0.46      0.49      0.48       115
weighted avg       0.86      0.91      0.89       115



In [60]:
# lbfgs, sgd, adam
#logistic, tanh, relu
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver="adam", activation="relu", learning_rate="constant", random_state=42, max_iter=3000, verbose=False).fit(X_trainScaledOS, y_trainOS)
clf.out_activation_="softmax"
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.06504065040650406
Confusion matrix: 
[[  0 107]
 [  0   8]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00       107
     class 1       0.07      1.00      0.13         8

    accuracy                           0.07       115
   macro avg       0.03      0.50      0.07       115
weighted avg       0.00      0.07      0.01       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=50, random_state=42).fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.6828271028037383
MCC: 0.5077102953881362
F1: 0.7362385321100917
Confusion matrix: 
[[106   1]
 [  5   3]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      0.99      0.97       107
     class 1       0.75      0.38      0.50         8

    accuracy                           0.95       115
   macro avg       0.85      0.68      0.74       115
weighted avg       0.94      0.95      0.94       115



In [62]:
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier().fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48198198198198194
Confusion matrix: 
[[107   0]
 [  8   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.93      1.00      0.96       107
     class 1       0.00      0.00      0.00         8

    accuracy                           0.93       115
   macro avg       0.47      0.50      0.48       115
weighted avg       0.87      0.93      0.90       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.5, max_depth=None, random_state=42).fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5391355140186915
MCC: 0.08954658875326628
F1: 0.5436507936507937
Confusion matrix: 
[[102   5]
 [  7   1]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.94      0.95      0.94       107
     class 1       0.17      0.12      0.14         8

    accuracy                           0.90       115
   macro avg       0.55      0.54      0.54       115
weighted avg       0.88      0.90      0.89       115



In [64]:
from xgboost import XGBClassifier
clf = XGBClassifier(max_depth=10, eta=0.3, reg_lambda=50, alpha=0).fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.5
MCC: 0.0
F1: 0.48198198198198194
Confusion matrix: 
[[107   0]
 [  8   0]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.93      1.00      0.96       107
     class 1       0.00      0.00      0.00         8

    accuracy                           0.93       115
   macro avg       0.47      0.50      0.48       115
weighted avg       0.87      0.93      0.90       115



C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fre_f\anaconda3\envs\FF\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
clf = LGBMClassifier().fit(X_trainScaledOS, y_trainOS)
clf.predict(X_testScaledOS)

metricsOS()

Balanced accuracy: 0.6203271028037383
MCC: 0.3841144978851179
F1: 0.6658364466583644
Confusion matrix: 
[[106   1]
 [  6   2]]
Classification report: 
              precision    recall  f1-score   support

     class 0       0.95      0.99      0.97       107
     class 1       0.67      0.25      0.36         8

    accuracy                           0.94       115
   macro avg       0.81      0.62      0.67       115
weighted avg       0.93      0.94      0.93       115

